In [ ]:
def loaddata():
    with open("./data/train-images.idx3-ubyte", 'rb') as f:

        magic_number = int.from_bytes(f.read(4), byteorder='big', signed=True)
        train_size = int.from_bytes(f.read(4), byteorder='big', signed=True)
        img_row = int.from_bytes(f.read(4), byteorder='big', signed=True)
        img_col  = int.from_bytes(f.read(4), byteorder='big', signed=True)
        
        train_data = [[[0 for j in range(img_row)]for i in range(img_col)]for k in range (train_size)]

        # a = [[0 for j in range(img_row)]for i in range(img_col)]
        for i in range (train_size) :
            for m in range(img_row):
                for n in range(img_row):
                    train_data [i][m][n] = int.from_bytes(f.read(1), byteorder='big', signed=True)
      

    with open("./data/train-labels.idx1-ubyte"  , 'rb') as f:

        magic_number = int.from_bytes(f.read(4), byteorder='big', signed=True)
        train_size  = int.from_bytes(f.read(4), byteorder='big', signed=True)
        
        train_label = [0 for j in range(train_size)]

        for i in range (train_size) :
            train_label[i] =  int.from_bytes(f.read(1), byteorder='big', signed=True) 

    with open("./data/t10k-images.idx3-ubyte" , 'rb') as f:

        magic_number = int.from_bytes(f.read(4), byteorder='big', signed=True)
        test_size = int.from_bytes(f.read(4), byteorder='big', signed=True)
        img_row = int.from_bytes(f.read(4), byteorder='big', signed=True)
        img_col  = int.from_bytes(f.read(4), byteorder='big', signed=True)
        
        test_data = [[[0 for j in range(img_row)]for i in range(img_col)] for k in range(test_size)]
        a = [[0 for j in range(img_row)]for i in range(img_col)]
       
        for i in range (test_size) :
            for m in range(img_row):
                for n in range(img_row):
                     test_data[i][m][n] = int.from_bytes(f.read(1), byteorder='big', signed=True)

    with open("./data/t10k-labels.idx1-ubyte"  , 'rb') as f:

        magic_number = int.from_bytes(f.read(4), byteorder='big', signed=True)
        test_size  = int.from_bytes(f.read(4), byteorder='big', signed=True)
        
        test_label = [0 for i in range(test_size)]
        for i in range (test_size) :
            test_label[i] = int.from_bytes(f.read(1), byteorder='big', signed=True) 
       
    
    return  train_data, train_label, test_data, test_label      
            
        


In [ ]:
train_data, train_label, test_data, test_label = loaddata()

In [ ]:
import math

# probablity
p = [[[[1.0 for n in range (28)] for m in range (28)] for j in range (32)] for i in range(10)]
prior = [0.0 for n in range (10)]

for k in range( len(train_data) ):
    prior[ train_label[k] ] += 1.0
    for i in range(28):
        for j in range(28):
            p[ train_label[k] ][ train_data[k][i][j]//8 ][i][j] += 1.0

prob = p.copy()
# test the probability
for i in range(10):
     for j in range(32):
        for m in range(28):
            for n in range(28):
                prob[i][j][m][n] /=  prior[i]

prior = [ prior[n] / 60000.0 for n in range (10)]


In [ ]:
posterior_log = [[0.0 for j in range (10)] for i in range(len(test_label))]
y_p = [ 0 for i in range (len(test_label))]
error = 0
for k in range(len(test_label)):
    max = 0  
    S = 0.0
    for n in range(10):
        # prior addition
        posterior_log[k][n] += math.log(prior[n])
        # point addition
        for i in range(28):
            for j in range(28):
                posterior_log[k][n] += math.log( prob[n][test_data[k][i][j]//8][i][j] )
        
        if (posterior_log[k][n] > posterior_log[k][max] ):
            max = n
        S += posterior_log[k][n]             
    for n in range(10):
        posterior_log[k][n] /= S
        # print(n, ":", posterior_log[k][n])
    y_p[k] = max
    
    if(y_p[k] != test_label[k]):
        error += 1  
    # print("Prediction:", max, "Ans:", test_label[k])    


print("error rate : ", error / len(y_p))          

In [ ]:
p_new = [[[[p[i][n][j][m] for n in range (32)] for m in range (28)] for j in range (28)] for i in range(10)]
max_a = [[[ 1.0 for m in range (28)] for j in range (28)] for i in range(10)]
for n in range(10):
    for i in range(28):
        for j in range(28):
            max = 0.0
            for k in range(32):
                if(p_new[n][i][j][k] > max):
                    max = p_new[n][i][j][k]
            max_a[n][i][j] = max    
                
print("Imagination of numbers in Bayesian classifier:")
for n in range(10):
    print(n, ":")
    for i in range(28):
        for j in range(28):
                if max_a[n][i][j] < 15:
                    print(0 , end = " ")
                else:
                    print(1 , end = " ")
        print()

In [ ]:
mean = [[[0 for k in range(28) ]for j in range(28)] for i in range (10)]
var = [[[1 for k in range(28) ]for j in range(28)] for i in range (10)]
count = [0 for i in range(10)]
prior = [0 for i in range(10)]


for i in range (len(train_label)):
    count[train_label[i]] +=1  

for n in range (len(train_data)):
    for i in range (28):
        for j in range(28):
            mean[train_label[n]][i][j] += train_data[n][i][j]
    prior[train_label[n]] += 1 


prior = [ prior[n] / 60000.0 for n in range (10)]

for n in range(10):
     for i in range (28):
        for j in range(28):
            mean[n][i][j] /= count[n]            

for n in range (len(train_data)):
    for i in range (28):
        for j in range(28):
            var[train_label[n]][i][j]+= (train_data[n][i][j] - mean[train_label[n]][i][j]) **2   
            

for n in range(10):
     for i in range (28):
        for j in range(28):
            var[n][i][j] /= count[n]  

  

print(mean[0])

In [ ]:
posterior_log = [[0.0 for j in range (10)] for i in range(len(test_label))]
y_p = [ [0] for i in range (len(test_data))]

for k in range(len(test_data)):
    max = 0  
    for n in range(10):
        posterior_log[k][n] += math.log10(prior[n])   # prior addition
        # point addition
        for i in range(28):
            for j in range(28):
                posterior_log[k][n] += (-0.5) * math.log10( 2 * math.pi * var[n][i][j] )
                posterior_log[k][n] += -0.5  * (((test_data[k][i][j]-mean[n][i][j]) **2)/ var[n][i][j])
        if (posterior_log[k][n] > posterior_log[k][max] ):
            max = n
    y_p[k] = max

# error
error = 0
for i in range (len(y_p)):
    if(y_p[i] != test_label[i]):
        error += 1  
print("error rate : ", error / len(y_p)) 

In [ ]:
for n in range (10):
    print(n , ":")
    for i in range(28):
        for j in range(28):
            if mean[n][i][j] < 128:
                print("0", end=" ")
            else:
                print("1", end=" ")
        print()
    print()    
